In [56]:
using Pkg
Pkg.add("JuMP")
Pkg.add("HiGHS")


   Resolving package versions...
  No Changes to `C:\Users\cooki\OneDrive\Documents\Ponts\2A\Rechop\Code\Project.toml`
  No Changes to `C:\Users\cooki\OneDrive\Documents\Ponts\2A\Rechop\Code\Manifest.toml`


   Resolving package versions...


  No Changes to `C:\Users\cooki\OneDrive\Documents\Ponts\2A\Rechop\Code\Project.toml`
  No Changes to `C:\Users\cooki\OneDrive\Documents\Ponts\2A\Rechop\Code\Manifest.toml`


In [57]:
using JuMP
using HiGHS

In [79]:
import .KIRO2023


chemin_tiny = "instances/KIRO-tiny.json"
#chemin_small = "instances/KIRO-small.json"
#chemin_medium = "instances/KIRO-medium.json"
#chemin_large = "instances/KIRO-large.json"
#chemin_huge = "instances/KIRO-huge.json"

current_instance = KIRO2023.read_instance(chemin_tiny)

nb_WT = length(current_instance.wind_turbines) #Nombre de wind_turbine dans notre instance : card(V^s)=card(E0)
nb_SS = length(current_instance.substation_locations) #Nombre de substation dans notre instance Card(V^t)
CardVS=nb_WT
CardE0=CardVS
CardVT=nb_SS
CardS = length(current_instance.substation_types) #Card(S)
CardE0=nb_WT
CardQ0=length(current_instance.land_substation_cable_types)
CardES= CardVS*(CardVS-1)
CardQS=length(current_instance.substation_substation_cable_types)
CardET=CardVT*CardVS

turb_links = zeros(Int,nb_WT) #Les trois tableaux suivants sont ceux qu'on va mettre dans notre type "solution" pour créer le json -> Z
st_cabl = zeros(Int,nb_SS) #partie E_0 de y
sub = Vector{KIRO2023.SubStation}() #Les substations qu'on va construire-> x et partie E_s de y

function donnevecteurs(x,y0,ys,z):
    turb_links = zeros(Int,nb_WT)
    st_cabl = zeros(Int,nb_SS)
    sub = Vector{KIRO2023.SubStation}()
    for i in 1:size(x, 1)  # Boucle pour les lignes
        for j in 1:size(x, 2)  # Boucle pour les colonnes
            println("Élément à la position ($i, $j) : ", x[i, j])
        end



KIRO2023.SubStation[]

In [49]:
function variables_binaires!(model, tableau)
    for i in eachindex(tableau)
        @variable(model, tableau[i], Bin)
    end
end

function contrainteQ1!(model,mu,alpha,beta,M) # Quand on met la contrainte sur mu, alpha, beta existent deja, donc pas besoin de les def, @variable(model,mu)
    @constraint(model, mu >= 0)
    @constraint(model, mu <= M * alpha)
    @constraint(model, mu <= beta)
    @constraint(model, mu >= beta - (1 - alpha) * M)
end

function contrainteQ2!(model,mu,beta,M,y) #Attention, dire avant que y est binaire avec @variable(model, y, Bin) @variable(model,mu)
    @constraint(model, -beta <= M * y)
    @constraint(model, beta <= M * (1 - y))
    @constraint(model, mu >= 0)
    @constraint(model, mu >= beta)
    @constraint(model, mu <= M * (1 - y))
    @constraint(model, mu <= beta + M * y)   
end

function contrainteQ3!(model, mu, alpha, beta,M,x,y) # @variable(model, y, Bin)  @variable(model, x) @variable(model, mu)
    contrainteQ2!(model,x,beta-alpha,2*M,y)
    @constraint(model, mu == beta - x)
end


function contrainteQ4!(model,alpha,beta,gamma,A,B,y) #déclarer @variable(model,y) et pareil avec alpha, beta, gamma : les 4 doivent exister en tant que variable avant
    for i in 1:size(A, 1)
        @constraint(model, sum(A[i, j] * [x, y, z][j] for j in 1:3) <= B[i])
        @constraint(model, y >= alpha+gamma)
        @constraint(model, y >= beta+gamma)
end

contrainteQ3! (generic function with 1 method)

In [55]:
"""M=10
alpha = 8
beta = 9
modeltest =  Model(HiGHS.Optimizer)
@variable(modeltest,y,Bin)
@variable(modeltest, x) 
@variable(modeltest,mu)
contrainteQ3!(modeltest,mu,alpha,beta,M,x,y)
@objective(modeltest,Min,mu)
optimize!(modeltest)"""

"M=10\nalpha = 8\nbeta = 9\nmodeltest =  Model(HiGHS.Optimizer)\n@variable(modeltest,y,Bin)\n@variable(modeltest, x) \n@variable(modeltest,mu)\ncontrainteQ3!(modeltest,mu,alpha,beta,M,x,y)\n@objective(modeltest,Min,mu)\noptimize!(modeltest)"